In [12]:
import asyncio
import zmq
from zmq.asyncio import Context
from dataclasses import dataclass, field, asdict
from typing import List
import datetime as dt
import json

from ipywidgets import widgets
from IPython.display import display

from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure

from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import DatetimeTickFormatter, LabelSet

In [13]:
output_notebook()

Loading BokehJS ...

In [14]:
DATA_STREAM_ENDPOINT = "tcp://127.0.0.1:5556"

In [15]:
LIMIT_POINTS = 50_000

In [16]:
async def periodic_render(handle, period=5, debug=False):
    while True:
        push_notebook(handle=handle)
        
        await asyncio.sleep(period)

In [17]:
def parse_msg(raw_msg: bytes):
    str_msg = raw_msg.decode("utf-8")
    split_ind = str_msg.index(" ")
    return str_msg[:split_ind], json.loads(str_msg[split_ind:])

In [18]:
@dataclass
class BookInfo:
    book_time: List = field(default_factory=list)
    bid: List = field(default_factory=list)
    ask: List = field(default_factory=list)
    mid: List = field(default_factory=list)

@dataclass
class SMAInfo:
    ma_time: List = field(default_factory=list)
    sma_short: List = field(default_factory=list)
    sma_long: List = field(default_factory=list)
    ema_short: List = field(default_factory=list)
    ema_long: List = field(default_factory=list)

# Show Graph
# Build Tracker
# Use ZMQ to feed data to chart

tracker = figure(plot_width=800, plot_height=400, x_axis_type='datetime')

BOOK_DATA = ColumnDataSource(data=asdict(BookInfo()))
tracker.step("book_time", "bid", source=BOOK_DATA, line_color="blue", legend="Bid")
tracker.step("book_time", "ask", source=BOOK_DATA, line_color="red", legend="Ask")
tracker.step("book_time", "mid", source=BOOK_DATA, line_color="green", legend="Mid")

MA_DATA = ColumnDataSource(data=asdict(SMAInfo()))
tracker.line("ma_time", "sma_short", source=MA_DATA, line_color="orange", legend="Sma Short")
tracker.line("ma_time", "sma_long", source=MA_DATA, line_color="gold", legend="Sma Long")

tracker.line("ma_time", "ema_short", source=MA_DATA, line_color="magenta", legend="Ema Short")
tracker.line("ma_time", "ema_long", source=MA_DATA, line_color="purple", legend="Ema Long")

tracker.legend.location = "top_left"
tracker.legend.click_policy = "hide"

STREAM_MAP = {
    "order_book": BOOK_DATA,
    "ma": MA_DATA,
}

In [19]:
# Set up static info grid
mid_tob_text = widgets.FloatText(description="Val:")
ema_short_text = widgets.FloatText(description="EMA:")
time_text = widgets.Text(description="Time:")
state_text = widgets.Textarea(description="State:", layout=widgets.Layout(height="100px", width="100%"))

static_grid = widgets.GridBox(
    [mid_tob_text, ema_short_text, time_text, state_text], 
    layout=widgets.Layout(grid_template_columns="90%"),
)

def update_static_grid(state):
    """
    {<key>:["value"], ...}
    """
    try:
        state_text.value = str(state)
        mid_tob_text.value = state["mid"][0]
        ema_short_text.value = state["ema_short"][0]
        
        ms_time = state['book_time'][0] / 1000
        time_text.value = f"{dt.datetime.fromtimestamp(ms_time):%Y/%m/%d %H:%M:%S}"
    except (KeyError, IndexError):
        pass  # Ignore if no data
    

In [20]:
async def data_listener(tracker_handle):
    
    # Build socket connection
    stat_source = Context().socket(zmq.SUB)
    stat_source.connect(DATA_STREAM_ENDPOINT)
    stat_source.setsockopt(zmq.SUBSCRIBE, b"")
    
    current_state = {}

    while True:
        raw_msg = await stat_source.recv()
        plot, msg = parse_msg(raw_msg)
        update_static_grid(current_state)
        current_state.update(msg)

        if plot not in STREAM_MAP:
            continue

        STREAM_MAP[plot].stream(msg, LIMIT_POINTS)
        

In [ ]:
tracker_handle = show(tracker, notebook_handle=True)
display(static_grid)

# Periodic chart refresh
render_task = asyncio.create_task(periodic_render(tracker_handle, period=3, debug=False))

try:
    await data_listener(tracker_handle)
except Exception as e:
    print(e)
finally:
    render_task.cancel()
    print("Closed")

GridBox(children=(FloatText(value=0.0, description='Val:'), FloatText(value=0.0, description='EMA:'), Text(val…

In [ ]:
# Develop strategy
# Historical analysis with sqlite
# Improve throttle design (jenga)
# Backtesting
# Infra
# Data Capture
# Deadman switch
# Hover and highlight values